PRUEBA 1: EVALUACIÓN INTEGRAL DE TECNOLOGÍAS SOLAR
"PV vs CSP"

1) Selección de Países y Recolección de Datos:
Esta celda procesa los datos del recurso solar y condiciones meteorológicas de Chile, Sudáfrica Y China.

Analisis de Recurso Solar

In [1]:
"""
PARAMETROS INICIALES:
Script para analizar archivos CSV de recurso solar (GHI, DHI, DNI)
para diferentes ubicaciones y generar gráficos resumen.
Lee archivos desde la carpeta ../Datos y guarda resultados en ../Resultados_Recurso_Solar.
"""
import polars as pl
import matplotlib.pyplot as plt
import os
import traceback # Para imprimir errores detallados

# --- Definición de Rutas y Ubicaciones ---

# Obtener el directorio del script de forma segura
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Si __file__ no está definido (ej. ejecución interactiva no estándar), usar CWD
    script_dir = os.getcwd()

# Asumiendo que este script está en PRUEBA1
datos_dir = os.path.abspath(os.path.join(script_dir, "Datos"))
# Directorio base para guardar resultados (se creará si no existe)
resultados_dir_base = os.path.abspath(os.path.join(script_dir, "graficos/recurso_solar"))
os.makedirs(resultados_dir_base, exist_ok=True)

print(f"Directorio del Script: {script_dir}")
print(f"Directorio de Datos: {datos_dir}")
print(f"Directorio Base de Resultados: {resultados_dir_base}")


# Definir las ubicaciones y sus archivos de datos únicos de la carpeta Datos
# Ahora solo especificamos el archivo, los metadatos se leerán del CSV.
ubicaciones = {
    'Chile': {
        'archivo': os.path.join(datos_dir, 'chile.csv') # Usar el original
    },
    'Espana': {
        'archivo': os.path.join(datos_dir, 'espana.csv')
    },
    'Australia': {
        'archivo': os.path.join(datos_dir, 'australia.csv')
    }
    # Antofagasta excluida
}

Directorio del Script: /home/nicole/proyecto/NicoleTorres/PRUEBA1
Directorio de Datos: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos
Directorio Base de Resultados: /home/nicole/proyecto/NicoleTorres/PRUEBA1/graficos/recurso_solar


In [2]:
# --- Funciones de Análisis y Gráficos ---

def analizar_estadisticas(df):
    """Calcula estadísticas descriptivas para columnas numéricas clave."""
    columnas_numericas = ['GHI', 'DHI', 'DNI', 'Temperature', 'Wind Speed']
    # Filtrar para incluir solo columnas numéricas que existen en el df
    columnas_a_describir = [col for col in columnas_numericas if col in df.columns and df[col].dtype in pl.NUMERIC_DTYPES]
    
    if not columnas_a_describir:
        print("  Advertencia: No se encontraron columnas numéricas adecuadas para estadísticas descriptivas.")
        return None
        
    print("  Calculando estadísticas descriptivas...")
    try:
        # Usar describe() para obtener estadísticas básicas
        stats_df = df.select(columnas_a_describir).describe(
             percentiles=[0.25, 0.5, 0.75]
        )
        return stats_df
    except Exception as e:
        print(f"  Error calculando estadísticas descriptivas: {e}")
        return None

def detectar_outliers_iqr(df, columna, factor_iqr=1.5):
    """Detecta outliers en una columna usando el método IQR."""
    if columna not in df.columns or df[columna].dtype not in pl.NUMERIC_DTYPES:
        return 0, 0, 0, None, None # Devolver ceros y None si la columna no es válida
        
    try:
        q1 = df[columna].quantile(0.25)
        q3 = df[columna].quantile(0.75)
        iqr = q3 - q1
        
        if iqr is None or q1 is None or q3 is None:
             print(f"  Advertencia: No se pudo calcular IQR para {columna} (posiblemente todos los valores son iguales o nulos).")
             return 0, 0, 0, None, None

        limite_inferior = q1 - factor_iqr * iqr
        limite_superior = q3 + factor_iqr * iqr
        
        # Contar outliers
        outliers_df = df.filter(
            (pl.col(columna) < limite_inferior) | (pl.col(columna) > limite_superior)
        )
        count_outliers = outliers_df.height
        
        # Calcular porcentaje
        total_validos = df[columna].is_not_null().sum()
        if total_validos > 0:
            porcentaje_outliers = (count_outliers / total_validos) * 100
        else:
            porcentaje_outliers = 0.0
            
        return count_outliers, porcentaje_outliers, total_validos, limite_inferior, limite_superior
        
    except Exception as e:
        print(f"  Error detectando outliers para {columna}: {e}")
        return 0, 0, df[columna].is_not_null().sum(), None, None # Devolver ceros pero intentar contar válidos

def resumen_outliers(df):
    """Genera un resumen de outliers para columnas numéricas clave."""
    print("  Detectando outliers (método IQR)...")
    columnas_numericas = ['GHI', 'DHI', 'DNI', 'Temperature', 'Wind Speed']
    resumen = {}
    for col in columnas_numericas:
        if col in df.columns:
             count, percent, total_validos, lim_inf, lim_sup = detectar_outliers_iqr(df, col)
             if total_validos > 0: # Solo reportar si hubo datos válidos para analizar
                 resumen[col] = {
                     'count': count,
                     'percentage': percent,
                     'lower_bound': lim_inf,
                     'upper_bound': lim_sup,
                     'total_valid': total_validos
                 }
             else:
                  print(f"  Columna {col} no tenía datos válidos para análisis de outliers.")
        else:
             print(f"  Advertencia: Columna {col} no encontrada para análisis de outliers.")
             
    return resumen

def analizar_irradiacion_anual(df):
    """Analiza la irradiación anual por tipo"""
    resultados = {}
    for col_name in ['GHI', 'DHI', 'DNI']:
        if col_name in df.columns:
            try:
                df_col = df.select(pl.col(col_name).cast(pl.Float64, strict=False).fill_null(0.0))
                resultados[col_name] = {
                    'promedio': float(df_col.mean()[0,0] or 0.0),
                    'maximo': float(df_col.max()[0,0] or 0.0),
                    'minimo': float(df_col.min()[0,0] or 0.0),
                    'total': float(df_col.sum()[0,0] or 0.0)
                }
            except Exception as e:
                print(f"  Error al procesar columna anual {col_name}: {e}")
                resultados[col_name] = {'promedio': 0.0, 'maximo': 0.0, 'minimo': 0.0, 'total': 0.0}
        else:
            print(f"  Advertencia: Columna {col_name} no encontrada para análisis anual.")
            resultados[col_name] = {'promedio': 0.0, 'maximo': 0.0, 'minimo': 0.0, 'total': 0.0}
    return resultados

def analizar_irradiacion_mensual(df):
    """Analiza la irradiación mensual"""
    if 'Month' not in df.columns:
        print("  Error: Columna 'Month' no encontrada para análisis mensual.")
        return pl.DataFrame()

    agg_exprs = []
    columnas_irradiacion = ['GHI', 'DHI', 'DNI']
    columnas_presentes = [col for col in columnas_irradiacion if col in df.columns]

    if not columnas_presentes:
         print("  Error: No hay columnas de irradiación (GHI, DHI, DNI) disponibles para análisis mensual.")
         return pl.DataFrame()

    for col_name in columnas_presentes:
        agg_exprs.append(pl.col(col_name).cast(pl.Float64, strict=False).fill_null(0.0).mean().alias(f'{col_name}_promedio'))

    try:
        mensual = df.group_by('Month').agg(agg_exprs).sort('Month')
        meses_completos = pl.DataFrame({'Month': range(1, 13)})
        mensual_completo = meses_completos.join(mensual, on='Month', how='left').fill_null(0.0)
    except Exception as e:
        print(f"  Error durante la agregación mensual: {e}")
        return pl.DataFrame()

    return mensual_completo

def crear_graficos(df, resultados_dir_ubicacion, resultados_anuales, resultados_mensuales, ubicacion):
    """Crea los gráficos de análisis para una ubicación específica"""
    os.makedirs(resultados_dir_ubicacion, exist_ok=True)
    print(f"  Guardando gráficos para {ubicacion} en '{resultados_dir_ubicacion}'.")

    # --- Gráfico Mensual ---
    if not resultados_mensuales.is_empty() and resultados_mensuales.height == 12:
        plt.figure(figsize=(12, 6))
        meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
        plot_legend_mensual = False
        if 'GHI_promedio' in resultados_mensuales.columns and resultados_mensuales['GHI_promedio'].sum() > 0:
            plt.plot(meses, resultados_mensuales['GHI_promedio'], label='GHI', marker='o')
            plot_legend_mensual = True
        if 'DHI_promedio' in resultados_mensuales.columns and resultados_mensuales['DHI_promedio'].sum() > 0:
             plt.plot(meses, resultados_mensuales['DHI_promedio'], label='DHI', marker='s')
             plot_legend_mensual = True
        if 'DNI_promedio' in resultados_mensuales.columns and resultados_mensuales['DNI_promedio'].sum() > 0:
            plt.plot(meses, resultados_mensuales['DNI_promedio'], label='DNI', marker='^')
            plot_legend_mensual = True

        plt.xlabel('Mes')
        plt.ylabel('Irradiancia Promedio (W/m²)')
        plt.title(f'Irradiancia Promedio Mensual en {ubicacion}')
        if plot_legend_mensual:
             plt.legend()
        else:
             plt.text(0.5, 0.5, 'No hay datos de irradiación válidos para mostrar',
                      horizontalalignment='center', verticalalignment='center',
                      transform=plt.gca().transAxes, fontsize=12, color='red')
        plt.grid(True)
        plt.tight_layout()
        try:
            plt.savefig(os.path.join(resultados_dir_ubicacion, f'irradiacion_mensual_{ubicacion}.png'))
            print(f"    Gráfico mensual guardado.")
        except Exception as e:
            print(f"    Error al guardar gráfico mensual: {e}")
        plt.close()
    else:
        print(f"  No se generó gráfico mensual para {ubicacion} (datos mensuales vacíos o incompletos: {resultados_mensuales.height} filas).")

    # --- Gráfico Horario ---
    if 'Hour' not in df.columns:
        print(f"  Advertencia: Columna 'Hour' no encontrada. No se puede generar gráfico horario para {ubicacion}.")
        return

    agg_exprs_horaria = []
    columnas_irradiacion = ['GHI', 'DHI', 'DNI']
    columnas_presentes_hr = [col for col in columnas_irradiacion if col in df.columns]

    if not columnas_presentes_hr:
        print(f"  Advertencia: No hay columnas de irradiación para gráfico horario de {ubicacion}.")
        return

    for col_name in columnas_presentes_hr:
        agg_exprs_horaria.append(pl.col(col_name).cast(pl.Float64, strict=False).fill_null(0.0).mean().alias(f'{col_name}_promedio'))

    try:
        horaria = df.group_by('Hour').agg(agg_exprs_horaria).sort('Hour')

        if not horaria.is_empty():
            plt.figure(figsize=(12, 6))
            horas = range(24)
            horas_df = pl.DataFrame({'Hour': range(24)})
            horaria_completa = horas_df.join(horaria, on='Hour', how='left').fill_null(0.0)

            plot_legend_horaria = False
            if 'GHI_promedio' in horaria_completa.columns and horaria_completa['GHI_promedio'].sum() > 0:
                plt.plot(horas, horaria_completa['GHI_promedio'], label='GHI', marker='o')
                plot_legend_horaria = True
            if 'DHI_promedio' in horaria_completa.columns and horaria_completa['DHI_promedio'].sum() > 0:
                 plt.plot(horas, horaria_completa['DHI_promedio'], label='DHI', marker='s')
                 plot_legend_horaria = True
            if 'DNI_promedio' in horaria_completa.columns and horaria_completa['DNI_promedio'].sum() > 0:
                plt.plot(horas, horaria_completa['DNI_promedio'], label='DNI', marker='^')
                plot_legend_horaria = True

            plt.xticks(range(0, 24, 1))
            plt.gca().set_xticklabels([f'{h:02d}:00' for h in range(24)])
            plt.xlabel('Hora del día (24h)')
            plt.ylabel('Irradiancia Promedio (W/m²)')
            plt.title(f'Distribución Horaria de la Irradiancia en {ubicacion}')
            if plot_legend_horaria:
                plt.legend()
            else:
                 plt.text(0.5, 0.5, 'No hay datos de irradiación válidos para mostrar',
                      horizontalalignment='center', verticalalignment='center',
                      transform=plt.gca().transAxes, fontsize=12, color='red')
            plt.grid(True)
            plt.tight_layout()
            try:
                plt.savefig(os.path.join(resultados_dir_ubicacion, f'distribucion_horaria_{ubicacion}.png'))
                print(f"    Gráfico horario guardado.")
            except Exception as e:
                print(f"    Error al guardar gráfico horario: {e}")
            plt.close()
        else:
             print(f"  No se generó gráfico horario para {ubicacion} (datos horarios vacíos).")

    except Exception as e:
        print(f"  Error durante la agregación horaria o graficación para {ubicacion}: {e}")

# --- Bloque Principal de Ejecución ---

def main():
    print("\n--- Iniciando Análisis de Recurso Solar ---")
    resultados_generales = {}
    
    # Directorio específico para el archivo TXT de resumen
    resultados_txt_dir = os.path.abspath(os.path.join(script_dir, "Resultados"))
    os.makedirs(resultados_txt_dir, exist_ok=True) # Crear si no existe
    
    # Definir el nombre del archivo de texto para el resumen completo
    archivo_resumen_txt = os.path.join(resultados_txt_dir, "analisis_estadistico_completo.txt")

    # Abrir el archivo de texto en modo escritura (sobrescribirá si existe)
    try:
        with open(archivo_resumen_txt, 'w', encoding='utf-8') as f_resumen:
            f_resumen.write("=== RESUMEN DE ANÁLISIS ESTADÍSTICO Y DE ANOMALÍAS ===\n\n")

            for nombre_ubicacion, datos_ubicacion in ubicaciones.items():
                print(f"\nProcesando ubicación: {nombre_ubicacion}")
                # Escribir encabezado de ubicación en el archivo de resumen
                f_resumen.write(f"=== {nombre_ubicacion} ===\n")
                archivo_csv = datos_ubicacion['archivo']
                lat, lon, elev, tz = None, None, None, None # Inicializar metadatos

                if not os.path.exists(archivo_csv):
                    print(f"  Error: Archivo no encontrado -> {archivo_csv}")
                    f_resumen.write(f"  Error: Archivo no encontrado -> {archivo_csv}\n\n")
                    continue

                try:
                    # --- Leer Metadatos --- 
                    print(f"  Leyendo metadatos de: {archivo_csv}")
                    with open(archivo_csv, 'r', encoding='utf-8') as f:
                        header_names_line = f.readline().strip()
                        header_values_line = f.readline().strip()
                    header_names = [h.strip() for h in header_names_line.split(',')] 
                    header_values = [v.strip() for v in header_values_line.split(',')] 
                    metadata_dict = dict(zip(header_names, header_values))
                    try:
                        lat = float(metadata_dict.get('Latitude', None))
                    except (TypeError, ValueError): pass
                    try:
                         lon = float(metadata_dict.get('Longitude', None))
                    except (TypeError, ValueError): pass
                    try:
                         elev = float(metadata_dict.get('Elevation', None))
                    except (TypeError, ValueError): pass
                    try:
                         tz = int(metadata_dict.get('Local Time Zone', None))
                    except (TypeError, ValueError): pass

                    print(f"  Metadatos extraídos: Lat={lat}, Lon={lon}, Elev={elev}, TZ={tz}")
                    f_resumen.write(f"  Archivo: {os.path.basename(archivo_csv)}\n")
                    f_resumen.write(f"  Metadatos: Lat={lat}, Lon={lon}, Elev={elev}, TZ={tz}\n")
                    datos_ubicacion['lat'] = lat
                    datos_ubicacion['lon'] = lon
                    datos_ubicacion['elev'] = elev
                    datos_ubicacion['tz'] = tz
                    # ----------------------------------------------------

                    # --- Leer Datos --- 
                    print(f"  Leyendo datos de: {archivo_csv}")
                    df = pl.read_csv(archivo_csv, skip_rows=2, infer_schema_length=10000, null_values=['NA', '', 'NULL'])
                    print(f"  Archivo leído. Columnas detectadas: {df.columns}")
                    print(f"  Número inicial de filas: {df.height}")
                    f_resumen.write(f"  Filas de datos leídas: {df.height}\n")

                    # --- Corrección Chile --- 
                    if nombre_ubicacion == 'Chile':
                        print("  Aplicando corrección específica para Chile (añadir Wind Speed y reordenar)...")
                        target_columns_chile = [
                            'Year', 'Month', 'Day', 'Hour', 'Minute',
                            'Temperature', 'DHI', 'GHI', 'Wind Direction', 'Wind Speed',
                            'DNI', 'Relative Humidity', 'Pressure', 'Solar Zenith Angle',
                            'Precipitable Water'
                        ]
                        if 'Wind Speed' not in df.columns:
                            print("    Columna 'Wind Speed' no encontrada. Intentando leerla por posición (índice 10).")
                            try:
                                df_wind = pl.read_csv(archivo_csv, skip_rows=2, has_header=False, use_cols=[10], new_columns=['Wind Speed'])
                                df = df.with_row_count()
                                df_wind = df_wind.with_row_count()
                                df = df.join(df_wind, on="row_nr").drop("row_nr")
                                print("    Columna 'Wind Speed' añadida.")
                            except Exception as wind_err:
                                print(f"    *** Error: No se pudo leer/añadir la columna 'Wind Speed' por posición: {wind_err} ***")
                        cols_existentes_para_chile = [col for col in target_columns_chile if col in df.columns]
                        if set(cols_existentes_para_chile) != set(target_columns_chile):
                            print(f"    Advertencia: Columnas finales para Chile después de corrección: {cols_existentes_para_chile}")
                        if not cols_existentes_para_chile:
                             print("    Error Crítico: No quedan columnas válidas para Chile después de la corrección.")
                             f_resumen.write("    Error Crítico: No quedan columnas válidas para Chile después de la corrección.\n\n")
                             continue
                        df = df.select(cols_existentes_para_chile)
                        print(f"    Columnas reordenadas para Chile: {df.columns}")
                    # --------------------------

                    # --- Análisis Estadístico y Anomalías --- 
                    estadisticas = analizar_estadisticas(df)
                    if estadisticas is not None:
                        print("\n  Estadísticas Descriptivas:")
                        print(estadisticas)
                        f_resumen.write("\n  Estadísticas Descriptivas:\n")
                        f_resumen.write(str(estadisticas) + "\n") # Convertir DataFrame a string
                    else:
                         f_resumen.write("\n  No se pudieron calcular estadísticas descriptivas.\n")
                    
                    resumen_anomalias = resumen_outliers(df)
                    if resumen_anomalias:
                        print("\n  Resumen de Anomalías (Outliers por IQR):")
                        f_resumen.write("\n  Resumen de Anomalías (Outliers por IQR):\n")
                        for col, data in resumen_anomalias.items():
                            linea_outlier = f"    {col}: {data['count']} outliers ({data['percentage']:.2f}%) [Límites: {data.get('lower_bound', 'N/A'):.2f} - {data.get('upper_bound', 'N/A'):.2f}] de {data['total_valid']} válidos"
                            print(linea_outlier)
                            f_resumen.write(linea_outlier + "\n")
                    else:
                         f_resumen.write("\n  No se detectaron outliers o no se pudo realizar el análisis.\n")
                    # ---------------------------------------------

                    # --- Análisis Irradiación y Gráficos --- 
                    print("\n  Realizando análisis de irradiación anual...")
                    resultados_anuales = analizar_irradiacion_anual(df)
                    print("  Realizando análisis de irradiación mensual...")
                    resultados_mensuales = analizar_irradiacion_mensual(df)

                    resultados_generales[nombre_ubicacion] = {
                        'metadata': datos_ubicacion,
                        'stats': estadisticas.to_dict(as_series=False) if estadisticas is not None else None, 
                        'outliers': resumen_anomalias,
                        'anual': resultados_anuales,
                        'mensual_df_polars': resultados_mensuales # Guardar el DF por si acaso
                    }

                    resultados_dir_ubicacion = os.path.join(resultados_dir_base, nombre_ubicacion)
                    print("\n  Creando gráficos...")
                    crear_graficos(df, resultados_dir_ubicacion, resultados_anuales, resultados_mensuales, nombre_ubicacion)
                    
                    print("\n  Resumen Anual Irradiación:")
                    f_resumen.write("\n  Resumen Anual Irradiación:\n")
                    for tipo, valores in resultados_anuales.items():
                         if valores: 
                             linea_anual = f"    {tipo}: Promedio={valores['promedio']:.2f}, Max={valores['maximo']:.2f}, Min={valores['minimo']:.2f}, Total={valores['total']:.0f} Wh/m² (aprox)"
                             print(linea_anual)
                             f_resumen.write(linea_anual + "\n")
                    # ---------------------------------------------

                except Exception as e:
                    print(f"*** Error procesando {nombre_ubicacion} ***")
                    print(f"  Archivo: {archivo_csv}")
                    print(f"  Error: {e}")
                    traceback.print_exc()
                    f_resumen.write(f"\n*** Error procesando {nombre_ubicacion}: {e} ***\n")
                
                # Separador entre ubicaciones en el archivo de texto
                f_resumen.write("\n" + "="*70 + "\n\n") 

            print(f"\n--- Análisis Completado --- Resumen guardado en: {archivo_resumen_txt}")

    except IOError as e:
         print(f"Error: No se pudo abrir o escribir en el archivo de resumen: {archivo_resumen_txt}")
         print(e)


print(f" Funciones definidas")

 Funciones definidas


In [3]:
if __name__ == "__main__":
    main() 


--- Iniciando Análisis de Recurso Solar ---

Procesando ubicación: Chile
  Leyendo metadatos de: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/chile.csv
  Metadatos extraídos: Lat=-20.63, Lon=-69.9, Elev=922.0, TZ=-4
  Leyendo datos de: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/chile.csv
  Archivo leído. Columnas detectadas: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'Relative Humidity', 'Precipitable Water', 'Wind Direction', 'Wind Speed', 'Pressure', 'Solar Zenith Angle', 'GHI', 'DHI', 'DNI']
  Número inicial de filas: 8760
  Aplicando corrección específica para Chile (añadir Wind Speed y reordenar)...
    Columnas reordenadas para Chile: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'DHI', 'GHI', 'Wind Direction', 'Wind Speed', 'DNI', 'Relative Humidity', 'Pressure', 'Solar Zenith Angle', 'Precipitable Water']
  Calculando estadísticas descriptivas...

  Estadísticas Descriptivas:
shape: (9, 6)
┌────────────┬────────────┬───────────┬────────────

/tmp/ipykernel_1925/3983873996.py:7: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  columnas_a_describir = [col for col in columnas_numericas if col in df.columns and df[col].dtype in pl.NUMERIC_DTYPES]
/tmp/ipykernel_1925/3983873996.py:26: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  if columna not in df.columns or df[columna].dtype not in pl.NUMERIC_DTYPES:


    Gráfico mensual guardado.
    Gráfico horario guardado.

  Resumen Anual Irradiación:
    GHI: Promedio=288.36, Max=1154.00, Min=0.00, Total=2526007 Wh/m² (aprox)
    DHI: Promedio=55.89, Max=527.00, Min=0.00, Total=489612 Wh/m² (aprox)
    DNI: Promedio=349.70, Max=1061.00, Min=0.00, Total=3063393 Wh/m² (aprox)

Procesando ubicación: Espana
  Leyendo metadatos de: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/espana.csv
  Metadatos extraídos: Lat=37.89, Lon=-4.78, Elev=138.0, TZ=1
  Leyendo datos de: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/espana.csv
  Archivo leído. Columnas detectadas: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'DHI', 'GHI', 'Wind Direction', 'Wind Speed', 'DNI', 'Relative Humidity', 'Pressure', 'Solar Zenith Angle', 'Precipitable Water']
  Número inicial de filas: 8760
  Calculando estadísticas descriptivas...

  Estadísticas Descriptivas:
shape: (9, 6)
┌────────────┬────────────┬───────────┬────────────┬─────────────┬────────────┐
│ 

/tmp/ipykernel_1925/3983873996.py:7: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  columnas_a_describir = [col for col in columnas_numericas if col in df.columns and df[col].dtype in pl.NUMERIC_DTYPES]
/tmp/ipykernel_1925/3983873996.py:26: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  if columna not in df.columns or df[columna].dtype not in pl.NUMERIC_DTYPES:


    Gráfico mensual guardado.
    Gráfico horario guardado.

  Resumen Anual Irradiación:
    GHI: Promedio=210.32, Max=1044.00, Min=0.00, Total=1842403 Wh/m² (aprox)
    DHI: Promedio=57.03, Max=524.00, Min=0.00, Total=499619 Wh/m² (aprox)
    DNI: Promedio=257.32, Max=997.00, Min=0.00, Total=2254116 Wh/m² (aprox)

Procesando ubicación: Australia
  Leyendo metadatos de: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/australia.csv
  Metadatos extraídos: Lat=-31.55, Lon=116.22, Elev=243.0, TZ=8
  Leyendo datos de: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Datos/australia.csv
  Archivo leído. Columnas detectadas: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'Dew Point', 'DHI', 'DNI', 'Surface Albedo', 'Pressure', 'Wind Speed', 'Solar Zenith Angle', 'GHI', 'Wind Direction']
  Número inicial de filas: 8760
  Calculando estadísticas descriptivas...

  Estadísticas Descriptivas:
shape: (9, 6)
┌────────────┬────────────┬──────────┬────────────┬─────────────┬────────────┐
│ sta

/tmp/ipykernel_1925/3983873996.py:7: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  columnas_a_describir = [col for col in columnas_numericas if col in df.columns and df[col].dtype in pl.NUMERIC_DTYPES]
/tmp/ipykernel_1925/3983873996.py:26: DeprecationWarning: `NUMERIC_DTYPES` is deprecated. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  if columna not in df.columns or df[columna].dtype not in pl.NUMERIC_DTYPES:


    Gráfico mensual guardado.
    Gráfico horario guardado.

  Resumen Anual Irradiación:
    GHI: Promedio=229.98, Max=1147.00, Min=0.00, Total=2014601 Wh/m² (aprox)
    DHI: Promedio=59.59, Max=800.00, Min=0.00, Total=521990 Wh/m² (aprox)
    DNI: Promedio=281.66, Max=1070.00, Min=0.00, Total=2467324 Wh/m² (aprox)

--- Análisis Completado --- Resumen guardado en: /home/nicole/proyecto/NicoleTorres/PRUEBA1/Resultados/analisis_estadistico_completo.txt


---------------------------------------------------------------------------------------------

2) Simulación de Plantas PV y CSP


Tecnología PV

In [22]:
#!/usr/bin/env python3
"""
Simulación de sistema fotovoltaico para múltiples países usando PySAM
Este script realiza simulaciones de sistemas fotovoltaicos para Chile, China y Sudáfrica
con diferentes capacidades de sistema
"""

import PySAM.Pvwattsv7 as pv
import PySAM.Lcoefcr as Lcoefcr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [23]:
# Configuración de parámetros del sistema
# ---------------------------------------
# Lista de países y sus archivos de recursos solares
paises = {
    "Chile": "datos_Chile.csv",
    "China": "datos_China.csv",
    "Sudafrica": "datos_Sudafrica.csv"
}

# Capacidades del sistema a simular (en kW)
capacidades = [500, 1000, 50000, 100000, 500000]

# Parámetros técnicos y económicos base
dc_ac_ratio = 1.2                 # Ratio DC/AC
tilt = 20                         # Inclinación de los paneles en grados
azimuth = 180                     # Azimut en grados (180 = orientación sur)
array_type = 1                    # 1 = montaje sobre techo con rack
gcr = 0.4                         # Ground Coverage Ratio
inv_eff = 96                      # Eficiencia del inversor (%)
losses = 14.0                     # Pérdidas del sistema (%)

# Parámetros económicos base
fixed_charge_rate = 0.07          # Tasa de carga fija (%)
fixed_operating_cost_base = 50_000     # Costo operativo fijo anual base ($/año)
variable_operating_cost = 0.01    # Costo operativo variable ($/kWh)

def calcular_costo_capital(capacidad_kw):
    """Calcula el costo de capital basado en la capacidad del sistema"""
    # Costo por kW disminuye con el tamaño del sistema
    costo_base_por_kw = 1000  # $/kW para sistemas pequeños
    factor_escala = 0.9  # Factor de reducción por escala
    costo_por_kw = costo_base_por_kw * (factor_escala ** np.log10(capacidad_kw/1000))
    return capacidad_kw * costo_por_kw

def calcular_costo_operativo_fijo(capacidad_kw):
    """Calcula el costo operativo fijo basado en la capacidad del sistema"""
    return fixed_operating_cost_base * (capacidad_kw / 1000) ** 0.8

def crear_directorio_resultados(pais, capacidad_kw):
    """Crea el directorio de resultados para un país y capacidad específica"""
    directorio = f"Resultados_{pais}/Capacidad_{capacidad_kw/1000:.0f}MW"
    if not os.path.exists(directorio):
        os.makedirs(directorio)
    return directorio

def simular_sistema_pv(pais, archivo_recurso, capacidad_kw):
    """Ejecuta la simulación del sistema fotovoltaico para un país y capacidad específica"""
    
    print(f"\n======== SIMULACIÓN PARA {pais} - {capacidad_kw/1000:.0f}MW ========")
    
    # Calcular costos específicos para esta capacidad
    capital_cost = calcular_costo_capital(capacidad_kw)
    fixed_operating_cost = calcular_costo_operativo_fijo(capacidad_kw)
    
    # Crear directorio de resultados
    directorio_resultados = crear_directorio_resultados(pais, capacidad_kw)
    
    # 1. Crear un modelo PVWatts
    print("Creando modelo PVWatts...")
    pv_model = pv.new()
    
    # 2. Asignar el archivo de recurso solar
    print(f"Usando archivo de recurso solar: {archivo_recurso}")
    pv_model.SolarResource.solar_resource_file = archivo_recurso
    
    # 3. Configurar los parámetros del sistema PV
    print("Configurando parámetros del sistema...")
    pv_model.SystemDesign.system_capacity = capacidad_kw
    pv_model.SystemDesign.dc_ac_ratio = dc_ac_ratio
    pv_model.SystemDesign.array_type = array_type
    pv_model.SystemDesign.azimuth = azimuth
    pv_model.SystemDesign.tilt = tilt
    pv_model.SystemDesign.gcr = gcr
    pv_model.SystemDesign.inv_eff = inv_eff
    pv_model.SystemDesign.losses = losses
    
    # 4. Ejecutar la simulación PVWatts
    print("Ejecutando simulación PVWatts...")
    pv_model.execute()
    
    # 5. Obtener los resultados
    print("Obteniendo resultados de la simulación...")
    annual_energy = pv_model.Outputs.annual_energy
    print(f"Generación anual de energía PV: {annual_energy:,.2f} kWh")
    
    # 6. Calcular el LCOE
    lcoe = calcular_lcoe(annual_energy, capital_cost, fixed_operating_cost)
    
    # 7. Guardar resultados en CSV
    guardar_resultados(annual_energy, lcoe, directorio_resultados, pais, capacidad_kw, capital_cost, fixed_operating_cost)
    
    # 8. Visualizar resultados mensuales
    visualizar_resultados_mensuales(pv_model, directorio_resultados, pais, capacidad_kw)
    
    # 9. Realizar análisis de sensibilidad
    analisis_sensibilidad_tilt(pv_model, directorio_resultados, pais, capacidad_kw)
    
    return pv_model, annual_energy, lcoe

def calcular_lcoe(annual_energy, capital_cost, fixed_operating_cost):
    """Calcula el LCOE (Levelized Cost of Energy)"""
    
    print("Calculando LCOE...")
    # Crear el modelo para cálculo de LCOE
    lcoe_model = Lcoefcr.new()

    # Asignar valores para el cálculo del LCOE
    lcoe_model.SimpleLCOE.annual_energy = annual_energy
    lcoe_model.SimpleLCOE.capital_cost = capital_cost
    lcoe_model.SimpleLCOE.fixed_charge_rate = fixed_charge_rate
    lcoe_model.SimpleLCOE.fixed_operating_cost = fixed_operating_cost
    lcoe_model.SimpleLCOE.variable_operating_cost = variable_operating_cost
    
    # Ejecutar el cálculo del LCOE
    lcoe_model.execute()
    
    # Obtener el LCOE calculado
    lcoe = lcoe_model.Outputs.lcoe_fcr
    print(f"LCOE: {lcoe:,.4f} $/kWh")
    
    return lcoe

def guardar_resultados(annual_energy, lcoe, directorio, pais, capacidad_kw, capital_cost, fixed_operating_cost):
    """Guarda los resultados en un archivo CSV"""
    
    print("Guardando resultados en CSV...")
    # Crear un DataFrame con los resultados
    df_results = pd.DataFrame({
        "Pais": [pais],
        "Capacidad_MW": [capacidad_kw/1000],
        "Annual_Energy_kWh": [annual_energy],
        "Capital_Cost_$": [capital_cost],
        "Fixed_Charge_Rate": [fixed_charge_rate],
        "Fixed_Operating_Cost_$/yr": [fixed_operating_cost],
        "Variable_Operating_Cost_$/kWh": [variable_operating_cost],
        "LCOE_$/kWh": [lcoe]
    })
    
    # Guardar en CSV
    ruta_archivo = os.path.join(directorio, f"resultados_{pais.lower()}_{capacidad_kw/1000:.0f}MW.csv")
    df_results.to_csv(ruta_archivo, index=False)
    print(f"Resultados guardados en {ruta_archivo}")

def visualizar_resultados_mensuales(pv_model, directorio, pais, capacidad_kw):
    """Visualiza la generación mensual de energía"""
    
    try:
        # Obtener generación mensual
        monthly_energy = pv_model.Outputs.monthly_energy
        
        # Crear gráfico de barras
        plt.figure(figsize=(10, 6))
        meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
        plt.bar(meses, monthly_energy)
        plt.title(f'Generación Mensual de Energía - {pais} - Sistema FV de {capacidad_kw/1000:.0f}MW')
        plt.xlabel('Mes')
        plt.ylabel('Energía (kWh)')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Guardar gráfico
        ruta_grafico = os.path.join(directorio, f'generacion_mensual_{pais.lower()}_{capacidad_kw/1000:.0f}MW.png')
        plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
        print(f"Gráfico de generación mensual guardado como '{ruta_grafico}'")
        
        plt.close()
    except Exception as e:
        print(f"No se pudo generar el gráfico: {e}")

def analisis_sensibilidad_tilt(pv_model, directorio, pais, capacidad_kw):
    """Realiza un análisis de sensibilidad variando la inclinación"""
    
    print(f"\nRealizando análisis de sensibilidad para la inclinación de los paneles en {pais}...")
    inclinaciones = [0, 10, 20, 30, 40]
    energias = []
    
    for tilt_value in inclinaciones:
        # Crear modelo para cada inclinación
        test_model = pv.new()
        test_model.SolarResource.solar_resource_file = pv_model.SolarResource.solar_resource_file
        
        # Configurar parámetros
        test_model.SystemDesign.system_capacity = capacidad_kw
        test_model.SystemDesign.dc_ac_ratio = dc_ac_ratio
        test_model.SystemDesign.array_type = array_type
        test_model.SystemDesign.azimuth = azimuth
        test_model.SystemDesign.tilt = tilt_value
        test_model.SystemDesign.gcr = gcr
        test_model.SystemDesign.inv_eff = inv_eff
        test_model.SystemDesign.losses = losses
        
        # Ejecutar simulación
        test_model.execute()
        
        # Obtener energía anual
        annual_energy = test_model.Outputs.annual_energy
        energias.append(annual_energy)
        print(f"  Inclinación: {tilt_value}°, Energía anual: {annual_energy:,.2f} kWh")
    
    # Crear gráfico
    plt.figure(figsize=(10, 6))
    plt.plot(inclinaciones, energias, marker='o', linestyle='-', linewidth=2)
    plt.title(f'Análisis de sensibilidad - Inclinación de paneles - {pais} - {capacidad_kw/1000:.0f}MW')
    plt.xlabel('Inclinación (grados)')
    plt.ylabel('Energía anual (kWh)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Guardar gráfico
    ruta_grafico = os.path.join(directorio, f'analisis_inclinacion_{pais.lower()}_{capacidad_kw/1000:.0f}MW.png')
    plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
    print(f"Gráfico de análisis de sensibilidad guardado como '{ruta_grafico}'")
    
    plt.close()

def comparar_capacidades(resultados):
    """Compara los resultados entre diferentes capacidades y genera gráficos comparativos"""
    
    print("\nGenerando comparación entre capacidades...")
    
    # Crear directorio para comparaciones
    directorio_comparacion = "Comparacion_Capacidades"
    if not os.path.exists(directorio_comparacion):
        os.makedirs(directorio_comparacion)
    
    # Definir colores para cada país
    colores = {'Chile': 'b', 'China': 'g', 'Sudafrica': 'r'}
    
    # Crear un gráfico que combine todos los países para energía vs capacidad
    plt.figure(figsize=(12, 8))
    
    for pais in paises.keys():
        # Preparar datos para comparación
        capacidades_mw = []
        energias = []
        
        for capacidad_kw in capacidades:
            if pais in resultados and capacidad_kw in resultados[pais]:
                capacidades_mw.append(capacidad_kw/1000)
                energias.append(resultados[pais][capacidad_kw]['annual_energy'])
        
        # Agregar curva al gráfico de energía
        plt.plot(capacidades_mw, energias, 
                color=colores[pais],
                marker='o', 
                linestyle='-', 
                linewidth=2,
                label=pais)
    
    # Configurar el gráfico de energía
    plt.title('Generación Anual de Energía vs Capacidad - Comparación entre Países')
    plt.xlabel('Capacidad (MW)')
    plt.ylabel('Energía anual (kWh)')
    plt.grid(True, alpha=0.3)
    plt.legend(title='País', loc='best')
    plt.tight_layout()
    
    # Guardar el gráfico de energía
    ruta_grafico = os.path.join(directorio_comparacion, 'energia_vs_capacidad_combinado.png')
    plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
    print(f"Gráfico de energía vs capacidad combinado guardado como '{ruta_grafico}'")
    
    plt.close()
    
    # Crear un gráfico que combine todos los países para LCOE vs capacidad
    plt.figure(figsize=(12, 8))
    
    for pais in paises.keys():
        # Preparar datos para comparación
        capacidades_mw = []
        lcoes = []
        
        for capacidad_kw in capacidades:
            if pais in resultados and capacidad_kw in resultados[pais]:
                capacidades_mw.append(capacidad_kw/1000)
                lcoes.append(resultados[pais][capacidad_kw]['lcoe'])
        
        # Agregar curva al gráfico de LCOE
        plt.plot(capacidades_mw, lcoes, 
                color=colores[pais],
                marker='o', 
                linestyle='-', 
                linewidth=2,
                label=pais)
    
    # Configurar el gráfico de LCOE
    plt.title('LCOE vs Capacidad - Comparación entre Países')
    plt.xlabel('Capacidad (MW)')
    plt.ylabel('LCOE ($/kWh)')
    plt.grid(True, alpha=0.3)
    plt.legend(title='País', loc='best')
    plt.tight_layout()
    
    # Guardar el gráfico de LCOE
    ruta_grafico = os.path.join(directorio_comparacion, 'lcoe_vs_capacidad_combinado.png')
    plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
    print(f"Gráfico de LCOE vs capacidad combinado guardado como '{ruta_grafico}'")
    
    plt.close()
    
    print(f"Gráficos de comparación guardados en el directorio '{directorio_comparacion}'")

def grafico_sensibilidad_combinado(resultados):
    """Genera un gráfico combinado de sensibilidad de inclinación para todas las capacidades de cada país"""
    
    print("\nGenerando gráficos combinados de sensibilidad de inclinación...")
    
    # Crear directorio para gráficos combinados
    directorio_combinado = "Graficos_Sensibilidad_Combinados"
    if not os.path.exists(directorio_combinado):
        os.makedirs(directorio_combinado)
    
    inclinaciones = [0, 10, 20, 30, 40]
    
    for pais in paises.keys():
        plt.figure(figsize=(12, 8))
        
        # Crear un gráfico para cada capacidad
        for capacidad_kw in capacidades:
            if pais in resultados and capacidad_kw in resultados[pais]:
                # Obtener datos de sensibilidad para esta capacidad
                energias = []
                for tilt_value in inclinaciones:
                    # Crear modelo para cada inclinación
                    test_model = pv.new()
                    test_model.SolarResource.solar_resource_file = paises[pais]
                    
                    # Configurar parámetros
                    test_model.SystemDesign.system_capacity = capacidad_kw
                    test_model.SystemDesign.dc_ac_ratio = dc_ac_ratio
                    test_model.SystemDesign.array_type = array_type
                    test_model.SystemDesign.azimuth = azimuth
                    test_model.SystemDesign.tilt = tilt_value
                    test_model.SystemDesign.gcr = gcr
                    test_model.SystemDesign.inv_eff = inv_eff
                    test_model.SystemDesign.losses = losses
                    
                    # Ejecutar simulación
                    test_model.execute()
                    
                    # Obtener energía anual
                    annual_energy = test_model.Outputs.annual_energy
                    energias.append(annual_energy)
                
                # Agregar curva al gráfico
                plt.plot(inclinaciones, energias, marker='o', linestyle='-', linewidth=2,
                        label=f'{capacidad_kw/1000:.0f}MW')
        
        # Configurar el gráfico
        plt.title(f'Análisis de Sensibilidad - Inclinación de Paneles - {pais}')
        plt.xlabel('Inclinación (grados)')
        plt.ylabel('Energía anual (kWh)')
        plt.grid(True, alpha=0.3)
        plt.legend(title='Capacidad del Sistema', loc='best')
        plt.tight_layout()
        
        # Guardar el gráfico
        ruta_grafico = os.path.join(directorio_combinado, f'sensibilidad_combinada_{pais.lower()}.png')
        plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
        print(f"Gráfico de sensibilidad combinada guardado como '{ruta_grafico}'")
        
        plt.close()

if __name__ == "__main__":
    print("======== SIMULACIÓN DE SISTEMAS FOTOVOLTAICOS PARA MÚLTIPLES PAÍSES Y CAPACIDADES ========")
    
    resultados = {}
    
    # Simular para cada país y cada capacidad
    for pais, archivo in paises.items():
        resultados[pais] = {}
        for capacidad_kw in capacidades:
            modelo, annual_energy, lcoe = simular_sistema_pv(pais, archivo, capacidad_kw)
            resultados[pais][capacidad_kw] = {
                'annual_energy': annual_energy,
                'lcoe': lcoe
            }
    
    # Generar comparación entre capacidades
    comparar_capacidades(resultados)
    
    # Generar gráficos combinados de sensibilidad
    grafico_sensibilidad_combinado(resultados)
    
    print("\n======== SIMULACIONES COMPLETADAS ========")
    print("Los resultados se han guardado en los respectivos directorios de cada país y capacidad")
    print("y se ha generado una comparación general en el directorio 'Comparacion_Capacidades'")
    print("Los gráficos combinados de sensibilidad se han guardado en el directorio 'Graficos_Sensibilidad_Combinados'") 

======== SIMULACIÓN DE SISTEMAS FOTOVOLTAICOS PARA MÚLTIPLES PAÍSES Y CAPACIDADES ========

======== SIMULACIÓN PARA Chile - 0MW ========
Creando modelo PVWatts...
Usando archivo de recurso solar: datos_Chile.csv
Configurando parámetros del sistema...
Ejecutando simulación PVWatts...


Exception: pvwattsv7 execution error.
	exec fail(pvwattsv7): could not open file for reading: datos_Chile.csv



Simulación CSP